In [4]:
import numpy as np
import torch
import torch.nn.functional as F
from prep import get_data
from torch import nn
from kfold_experiments import kfold_normal_training_softgedt, kfold_adversarial_training_softgedt,kfold_normal_training_neural_networks, kfold_adversarial_training_neural_networks

# Create dataset

In [5]:
# load dataset
filename = 'wdbc'
# Preprocess the data without standardizing
data, ncat,dtypes = get_data(filename)

# create editability vector
non_editable_vector = torch.tensor([1 for i in range(data[:,:-1].shape[1])])

# record datatypes
dtypes[dtypes=='object'] = 'pos_int'
dtypes[dtypes=='float64'] = 'float'

datatypes = dtypes[:-1].to_numpy().astype('object')
datatypes[(datatypes=='int64') & np.min(data[:,:-1]>=0,axis=0)]='pos_int'
datatypes[(datatypes=='float64') & np.min(data[:,:-1]>=0,axis=0)]='pos_float'
datatypes = datatypes.tolist()

# Normal Training

In [6]:
kfold_normal_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight = True)

experiment: 0
Fold: 0
epoch: 0
train acc: tensor(0.9912) valid acc: tensor(0.9298) trigger times: 0
epoch: 1
train acc: tensor(0.9912) valid acc: tensor(0.9298) trigger times: 0
epoch: 2
train acc: tensor(0.9912) valid acc: tensor(0.9298) trigger times: 1
epoch: 3
train acc: tensor(0.9941) valid acc: tensor(0.9298) trigger times: 2
epoch: 4
train acc: tensor(0.9941) valid acc: tensor(0.9298) trigger times: 3
Early Stopping
SoftGeDT Accuracy on Training set: tensor(0.9941)
SoftGeDT Accuracy on Validation set: tensor(0.9298)
SoftGeDT Accuracy on Test set: tensor(0.9825)
Average Accuracy on Test set: tensor(0.9825)
Original Accuracy on Test set: tensor(0.9825)
Average Original Accuracy on Test set: tensor(0.9825)
Original HardGeDT Accuracy on Test set: tensor(0.9386)
Average Original HardGeDT Accuracy on Test set: tensor(0.9386)
ADT error, epsilon:0.1: 0.07017543859649122
Average ADT error, epsilon:0.1 0.07017543859649122
HardGeDT Adversarial Error: tensor(0.0965)
HardGeDT Average Adversa

# Adversarial Training

In [5]:
kfold_adversarial_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight=True)

Fold: 0
epoch: 0
train error: 0.07624633431085044 train loss: 0.49331557505990753 validation error 0.05263157894736842
epoch: 1
train error: 0.07917888563049853 train loss: 0.2761322745829384 validation error 0.07017543859649122
epoch: 2
train error: 0.06744868035190615 train loss: 0.20484992031477764 validation error 0.06140350877192982
epoch: 3
train error: 0.05278592375366569 train loss: 0.14931108385237082 validation error 0.043859649122807015
epoch: 4
train error: 0.04398826979472141 train loss: 0.13284498313305315 validation error 0.043859649122807015
epoch: 5
train error: 0.0469208211143695 train loss: 0.1247865447550575 validation error 0.043859649122807015
epoch: 6
train error: 0.04985337243401759 train loss: 0.12940317969168386 validation error 0.043859649122807015
epoch: 7
train error: 0.03812316715542522 train loss: 0.12427251010346622 validation error 0.043859649122807015
Early Stopping
FNN Adversarial Error on Training set: 0.04985337243401759
FNN Adversarial Error on Val

# Simple Neural Network

In [4]:
class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim, 80).double()
        self.layer2 = nn.Linear(80, 80).double()
        self.layer3 = nn.Linear(80, 80).double()
        self.layer4 = nn.Linear(80, 2).double()
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = F.relu(self.dropout(self.layer1(x)))
        x = F.relu(self.dropout(self.layer2(x)))
        x = F.relu(self.dropout(self.layer3(x)))
        x = self.layer4(x)
        return x

## 5-fold cross validation normal training

In [6]:
kfold_normal_training_neural_networks(nn, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train acc: tensor(0.8798) valid acc: tensor(0.9649) trigger times: 0
epoch: 1
train acc: tensor(0.9765) valid acc: tensor(0.9649) trigger times: 0
epoch: 2
train acc: tensor(0.9853) valid acc: tensor(0.9474) trigger times: 1
epoch: 3
train acc: tensor(0.9912) valid acc: tensor(0.9561) trigger times: 2
epoch: 4
train acc: tensor(0.9912) valid acc: tensor(0.9649) trigger times: 0
epoch: 5
train acc: tensor(0.9971) valid acc: tensor(0.9649) trigger times: 0
epoch: 6
train acc: tensor(0.9941) valid acc: tensor(0.9474) trigger times: 1
epoch: 7
train acc: tensor(0.9971) valid acc: tensor(0.9649) trigger times: 2
epoch: 8
train acc: tensor(0.9971) valid acc: tensor(0.9561) trigger times: 0
epoch: 9
train acc: tensor(0.9912) valid acc: tensor(0.9649) trigger times: 1
epoch: 10
train acc: tensor(0.9971) valid acc: tensor(0.9561) trigger times: 0
epoch: 11
train acc: tensor(0.9941) valid acc: tensor(0.9561) trigger times: 1
epoch: 12
train acc: tensor(0.9941) valid acc: tensor(

KeyboardInterrupt: 

## 5 fold cross validation adversarial training

In [8]:
kfold_adversarial_training_neural_networks(Net, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train error: 0.13196480938416422 train loss: 0.3435834207248264 validation error 0.07894736842105263
epoch: 1
train error: 0.05865102639296188 train loss: 0.21784118627727508 validation error 0.043859649122807015
epoch: 2
train error: 0.05571847507331378 train loss: 0.12323723317100603 validation error 0.07894736842105263
epoch: 3
train error: 0.04398826979472141 train loss: 0.10554969647047314 validation error 0.06140350877192982
epoch: 4
train error: 0.03519061583577713 train loss: 0.08492749881976512 validation error 0.06140350877192982
epoch: 5
train error: 0.03519061583577713 train loss: 0.07190588927737072 validation error 0.06140350877192982
epoch: 6
train error: 0.02346041055718475 train loss: 0.06241706587169753 validation error 0.06140350877192982
epoch: 7
train error: 0.03225806451612903 train loss: 0.06240764428015512 validation error 0.06140350877192982
Early Stopping
FNN Adversarial Error on Training set: 0.02346041055718475
FNN Adversarial Error on Valid